<center><img alt="" src="images/covers/taudata-cover.jpg"/></center> 

## <center><font color="blue">Pendahuluan Model Rekomendasi: Market Basket Analysis/Association Rule</font></center>

<h2 style="text-align: center;">(C) Taufik Sutanto</h2>
<h2 style="text-align: center;"><a href="https://taudata.blogspot.com/2021/04/adm-03.html" target="_blank"><span style="color: #0009ff;">https://taudata.blogspot.com/2021/04/adm-03.html</span></a></h2>

In [ ]:
# Warning install "mlxtend" module first. Jalankan cell ini Hanya jika menggunakan Google Colaboratory:
# Jika menggunakan Jupyter notebook secara lokal (e.g. Anaconda/WinPython) silahkan install lewat terminal (e.g. command prompt)
try:
    import google.colab as gc_
    print("Running the code in Google Colab.", gc_) #IN_COLAB = True
    print("Installing required Module, please wait ... ")
    !pip install mlxtend 
    !pip install pycaret
except:
    print("Running the code locally, make sure to install required modules.")#IN_COLAB = False

In [ ]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from itertools import combinations
from collections import Counter
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pycaret.arules import *

%matplotlib inline
plt.style.use('bmh'); sns.set()

In [ ]:
# In Python
T = [
 ('Bread', 'Milk'),
 ('Beer', 'Bread', 'Diaper', 'Eggs', 'Milk', 'Bread', 'Milk', 'Milk'),
 ('Beer', 'Coke', 'Diaper', 'Milk'),
 ('Beer', 'Bread', 'Diaper', 'Milk'),
 ('Bread', 'Coke', 'Diaper', 'Milk', 'Diaper'),
]
T

In [ ]:
# Calculating item sets
# Nostalgia Matematika Diskrit :)
def subsets(S, k):
    return [set(s) for s in combinations(S, k)]

subsets({1, 2, 3, 7, 8}, 3)

In [ ]:
# Calculating support
Counter(T[1])

In [ ]:
# Using Module
# Taken from https://pbpython.com/market-basket-analysis.html
# Pertama-tama load Data
try:
    df = pd.read_csv('data/Online_Retail.csv', error_bad_lines=False, low_memory = False)
except:
    df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

In [ ]:
# Preprocessing
df['Description'] = df['Description'].str.strip() # remove unnecessary spaces
df['Description'] = df['Description'].str.lower() # lower case normalization
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # delete rows with no invoice no
df['InvoiceNo'] = df['InvoiceNo'].astype('str') # Change data type
df = df[~df['InvoiceNo'].str.contains('c')] # remove invoice with C in it
df.head()

In [ ]:
df.to_csv("data/Online_Retail.csv", encoding='utf8', index=False)
'Done'

In [ ]:
filter_ = {'pls', 'plas'}
for f in filter_:
    df = df[~df['InvoiceNo'].str.contains(f)] # filtering invoice

In [ ]:
print(set(df['Country']))

In [ ]:
df_A = df[df['Country'] =="Australia"]
df_A.head()

In [ ]:
# Let's sample the data
basket = df[df['Country'] =="Australia"]
basket.head()

In [ ]:
# Group the transaction
basket = basket.groupby(['InvoiceNo', 'Description'])['Quantity']
basket.head()

In [ ]:
# Jumlahkan, unstack, Null=0, index baris menggunakan Nomer Invoice
basket = basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket.head()

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units) # one-hot encoding
basket_sets.head()

# Understanding the Data Structure

In [ ]:
basket_sets.columns

In [ ]:
basket_sets.index

In [ ]:
basket_sets.iloc[0]

In [ ]:
basket_sets.iloc[0].sum()

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, na_position='last', inplace = True)
frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values(by='lift', ascending=False, na_position='last', inplace = True)
rules.head(5)

In [ ]:
# Filtering
rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ]

# End of Module Association Rule
<hr>
<p><img alt="" src="images/meme-cartoon/10_recommendation_joke.jpg" style="height:445px; width:455px" /></p>
